# Song recommendation in action
***
Here we demonstrated an example of song recommendation using NNCF model.
***
### Load libraries

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import scipy.sparse as sps
import scipy
import pickle
import keras
from keras.models import load_model

Using TensorFlow backend.


### Connect to database

In [12]:
# connect to database
conn = sqlite3.connect("spotifyDB.db")

### Load data and model

In [2]:
# load files
sps_acc = scipy.sparse.load_npz('sparse_10000.npz')
with open('sublist.pkl', 'rb') as f1:
    sublist = pickle.load(f1)
with open('tracks.pkl', 'rb') as f2:
    tracks = pickle.load(f2)

# convert matrix to csc type
sps_acc = sps_acc.tocsc()
sps_acc

<74450x10000 sparse matrix of type '<class 'numpy.float64'>'
	with 1030015 stored elements in Compressed Sparse Column format>

In [3]:
# load the best NNCF model
model_file = "models_10000/model.220-0.30-0.88.h5"
model = load_model(model_file)

# find track and playlist number
n_tracks, n_playlists = sps_acc.shape[0], sps_acc.shape[1]

### Pull out function for recommendation

In [4]:
# this function returns the indices and scores for recommeded tracks given a playlist_id
def recommend_tracks(sps_acc, playlist_ind, model):
    pairs = [np.ones((n_tracks,))*playlist_ind, np.arange(n_tracks)]

    # make prediction with the model
    prob = model.predict(pairs)
    pred_class = prob.argmax(axis=-1)

    # extract real class label from original data
    real_class = np.array(sps_acc[:,playlist_ind].toarray()).reshape(-1,)
    
    # real tracks: real class one
    real_one_ind = np.argwhere(real_class).reshape(-1,)
    
    # recommended tracks: predicted class one but real class zero
    recom = np.logical_and(pred_class == 1, real_class == 0)

    # find recommeded track index and score of being one
    recom_ind = np.argwhere(recom).reshape(-1,)
    recom_prob = prob[recom_ind,1]

    # sort recommended indices from highest score to lowest
    recom_sort_ind = np.argsort(recom_prob)
    recom_ind = recom_ind[recom_sort_ind[::-1]]

    # not-recommended tracks: predicted class zero and real class zero
    not_recom = np.logical_and(pred_class == 0, real_class == 0)

    # find not-recommeded track index and score of being zero
    not_recom_ind = np.argwhere(not_recom).reshape(-1,)
    not_recom_prob = prob[not_recom_ind,0]

    # sort not-recommended indices from highest score to lowest
    not_recom_sort_ind = np.argsort(not_recom_prob)
    not_recom_ind = not_recom_ind[not_recom_sort_ind[::-1]]

    return recom_ind, not_recom_ind, real_one_ind

## Make song recommendations to a popular playlist
***
Here we identified the most popular playlist in the subset by searching the playlist with maximal number of followers. It was a playlist called No Limit with 212 tracks in it and had 232 followers.

In [101]:
keys = [str(pl) for pl in sublist]
keys = '\',\''.join(keys)
keys = "('"+keys+"')"
    
# fetch information of first 10 popular playlists
query = 'SELECT playlist_id,playlist_name,num_tracks, num_followers \
FROM playlists WHERE playlist_id IN {} ORDER BY num_followers DESC LIMIT 10;'.format(keys)
playlist_df = pd.read_sql_query(query, conn)
display(playlist_df)

,playlist_id,playlist_name,num_tracks,num_followers
0,276338,No Limit,212,232
1,126788,No Heart,116,227
2,148574,Needed Me,230,90
3,12870,Desiigner — Panda,184,87
4,306214,Shabba,209,76
5,220058,Old playlist.,242,74
6,274306,No Limit,25,68
7,187065,🔥lit🔥,167,58
8,90643,bonfire playlist,243,32
9,271904,Hip Hop,51,27


Now we make recommendation to this playlist using our NNCF model.

In [105]:
# find the corresponding position of the playlist in our subset
this_id = sublist.index(playlist_df.playlist_id.values[0])

# make recommendation using NNCF model
rec, _, real = recommend_tracks(sps_acc, this_id, model)

Here are the first 20 songs in this playlist.

In [106]:
# obtain track uri for the first 20 tracks in the playlist
member = [tr for idx, tr in enumerate(tracks) if idx in list(real[:20])]
keys = '\',\''.join(member)
keys = "('"+keys+"')"
    
# fetch information of existing tracks
query = 'SELECT track_name, album_name, artist_name FROM tracks WHERE track IN {};'.format(keys)
display(pd.read_sql_query(query, conn))

,track_name,album_name,artist_name
0,Frontin' - Club Mix,The Neptunes Present... Clones,Pharrell Williams
1,R.O.C.K. In The U.S.A. (A Salute To 60's Rock),Scarecrow,John Mellencamp
2,No Te Quiero En Mi Vida,1392,David Lee Garza
3,No Money (Remix),No Money (Remix),Vapi
4,7 Oaks,The Turnpike Troubadours,Turnpike Troubadours
5,Sly Fox - Original Mix,The Adventures of Mr. Fox,Koan Sound
6,Bonbon - English Version,Bonbon,Era Istrefi
7,Fancy - Yellow Claw Remix,Fancy,Iggy Azalea
8,Thugged Out - feat. Foxx [Explicit Album Version],Incarcerated,Boosie Badazz
9,Winner Take All,The Pursuit of Nappyness,Nappy Roots


Here are the top 10 songs that the model recommended!

In [107]:
# obtain track uir for the top 10 recommended songs 
rec_tracks = [tr for idx, tr in enumerate(tracks) if idx in list(rec[:10])]
keys = '\',\''.join(rec_tracks)
keys = "('"+keys+"')"
    
# fetch information of recommended songs
query = 'SELECT track_name, album_name, artist_name FROM tracks WHERE track IN {};'.format(keys)
display(pd.read_sql_query(query, conn))

,track_name,album_name,artist_name
0,Call Me,Call Me,NEIKED
1,Almeria,Almeria,Everydayz
2,Where Is The Love?,Elephunk,The Black Eyed Peas
3,On verra,Feu,Nekfeu
4,Jolene,The Foundation,Zac Brown Band
5,Knotty Head,Imperial,Denzel Curry
6,Strangeland,All 6's And 7's,Tech N9ne
7,Seeing Two,Soda Lime Love,MIAMIGO
8,You Don't Know Me (feat. Hemi),Slow Motion,Jarren Benton
9,Maca.Frama.Lamma,Da U.S. Open,Mac Dre And Mac Mall


In [108]:
conn.close()